In [11]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

# Define input sizes for each model
input_sizes = {
    "Custom CNN": (224, 224),
    "MobileNetV2": (224, 224),
    "EfficientNetB0": (224, 224),
    "ResNet50": (299, 299),
    "InceptionV3": (299, 299)
}

# List of saved models
model_files = {
    "Custom CNN": "cnn_model.h5",
    "MobileNetV2": "mobinet_model.h5",
    "EfficientNetB0": "eff_model.h5",
    "ResNet50": "restnet_model.h5",
    "InceptionV3": "inceptionV3_model.h5"
}

# Class names (replace with your actual class list)
class_names = [
    'animal fish', 'animal fish bass', 'fish sea_food black_sea_sprat',
    'fish sea_food gilt_head_bream', 'fish sea_food hourse_mackerel',
    'fish sea_food red_mullet', 'fish sea_food red_sea_bream',
    'fish sea_food sea_bass', 'fish sea_food shrimp',
    'fish sea_food striped_red_mullet', 'fish sea_food trout'
]

In [12]:
results = []

# Loop through each model and evaluate
for model_name, file_path in model_files.items():
    print(f"\nEvaluating {model_name}...")

    # Create test generator for the specific model input size
    target_size = input_sizes[model_name]
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_gen = test_datagen.flow_from_directory(
        'data/test',  # Change to your actual test dataset path
        target_size=target_size,
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )

    # Load model
    model = tf.keras.models.load_model(file_path)

    # Evaluate accuracy
    loss, acc = model.evaluate(test_gen, verbose=0)

    # Predictions
    y_pred = np.argmax(model.predict(test_gen), axis=1)
    y_true = test_gen.classes

    # Classification report
    report = classification_report(
        y_true, y_pred, target_names=class_names, output_dict=True
    )
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    f1 = report['weighted avg']['f1-score']

    results.append({
        "Model": model_name,
        "Accuracy": acc,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    })

# Create DataFrame for easy comparison
df_results = pd.DataFrame(results).sort_values(by="Accuracy", ascending=False).reset_index(drop=True)

print("\n=== Model Comparison ===")
print(df_results)

# Champion model
champion = df_results.iloc[0]
print(f"\n🏆 Champion Model: {champion['Model']} with Accuracy: {champion['Accuracy']:.4f}")



Evaluating Custom CNN...
Found 3187 images belonging to 11 classes.


100/100 ━━━━━━━━━━━━━━━━━━━━ 14s 137ms/step

Evaluating MobileNetV2...
Found 3187 images belonging to 11 classes.


100/100 ━━━━━━━━━━━━━━━━━━━━ 50s 485ms/step

Evaluating EfficientNetB0...
Found 3187 images belonging to 11 classes.


100/100 ━━━━━━━━━━━━━━━━━━━━ 66s 638ms/step 

Evaluating ResNet50...
Found 3187 images belonging to 11 classes.


100/100 ━━━━━━━━━━━━━━━━━━━━ 248s 2s/step

Evaluating InceptionV3...
Found 3187 images belonging to 11 classes.


100/100 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step

=== Model Comparison ===
            Model  Accuracy  Precision    Recall  F1-Score
0     MobileNetV2  0.992783   0.992913  0.992783  0.992052
1     InceptionV3  0.980546   0.981345  0.980546  0.980128
2      Custom CNN  0.865391   0.873671  0.865391  0.856002
3        ResNet50  0.215249   0.101059  0.215249  0.122535
4  EfficientNetB0  0.163163   0.026622  0.163163  0.045775

🏆 Champion Model: MobileNetV2 with Accuracy: 0.9928


# StreamLit Web App